# Demo: Cortex Agents Python API (in Snowflake Notebooks)

Make sure your notebook has an External Access Integration 

Create your External Access Integration like this:
```sql
CREATE OR REPLACE NETWORK RULE self_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('<ORGNAME>-<ACCOUNTNAME>.snowflakecomputing.com');

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION self_access_integration
  ALLOWED_NETWORK_RULES = (self_rule)
```

In [ ]:
# Define log level
from cortex_agent import setup_module_logger
setup_module_logger(level='WARN')

# Imports
from snowflake.snowpark import Session
from cortex_agent.agent import CortexAgent
from cortex_agent.configuration import CortexAgentConfiguration
from cortex_agent.tools import (
    CortexSearchTool, 
    CortexAnalystTool, 
    SQLExecTool,
    DataToChartTool
    )
from cortex_agent.tool_resources import CortexSearchService, CortexAnalystService

# Create a session
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Create an Agent Configuration
An agent has access to one or more tools.  

In [ ]:
from cortex_agent.configuration import CortexAgentConfiguration
from cortex_agent.tools import SQLExecTool, CortexSearchTool, CortexAnalystTool, DataToChartTool

# Basic Tools
sql_exec_tool = SQLExecTool(name='sql_exec1')
data_to_chart_tool = DataToChartTool(name='datatochart1')

# Cortex Analyst Tool and Tool Resources
analyst_tool = CortexAnalystTool(name='analyst1')
analyst_tool_resource = CortexAnalystService(resource_name='analyst1', semantic_model_file='@CORTEX_AGENTS_DEMO.main.semantic_models/sales_orders.yaml')

# Cortex Search Tool and Tool Resources
search_tool = CortexSearchTool(name='search1')
search_tool_resource = CortexSearchService(
    resource_name='search1', 
    database='CORTEX_AGENTS_DEMO', 
    schema='MAIN', 
    service_name='ANNUAL_REPORTS_SEARCH', 
    max_results=2, 
    title_column='RELATIVE_PATH', 
    id_column='CHUNK_INDEX'
    )

# Adding all tools to the Configuration
agent_config = CortexAgentConfiguration(
    model='claude-3-5-sonnet',
    tools=[sql_exec_tool, data_to_chart_tool, analyst_tool,search_tool], 
    tool_resources=[analyst_tool_resource, search_tool_resource]
    )

print(agent_config)

### Agent with Programmatic Access Token
You can use a Programmatic Access Token to connect your Agent with Snowflake.  

In [ ]:
# Create a new token for your user
agent_api_token = session.sql("ALTER USER ADD PROGRAMMATIC ACCESS TOKEN agent_api_token3").collect()[0]['token_secret']

In [ ]:
agent_api_token = open('pat.txt').readline()
connection_parameters = {
    "account": "SFSEEUROPE-PROD-DEMO-GORKOW.snowflakecomputing.com",
    "user": "ADMIN",
    "role": "ACCOUNTADMIN",
    "warehouse": "COMPUTE_WH",
    "database": "cortex_agents_demo",
    "schema": "MAIN",
}

agent = CortexAgent(
    programmatic_access_token=agent_api_token, 
    connection_parameters=connection_parameters, 
    configuration=agent_config, 
    session=session
)

for chunk in agent.make_request(content='What were the latest AI innovations from Googol in 2024?'):
    print(chunk, end='')

FileNotFoundError: [Errno 2] No such file or directory: 'pat.txt'

### Agent with Private Key File
You can use a Private Key File to connect your Agent with Snowflake.  
Make sure your key is accessible in the notebook.

In [ ]:
connection_parameters = {
    "account": "SFSEEUROPE-PROD-DEMO-GORKOW.snowflakecomputing.com",
    "user": "ADMIN",
    "role": "ACCOUNTADMIN",
    "warehouse": "COMPUTE_WH",
    "database": "CORTEX_AGENTS_DEMO",
    "schema": "MAIN"
}

agent = CortexAgent(
    private_key_file='rsa_key.p8', 
    connection_parameters=connection_parameters, 
    configuration=agent_config, 
    session=session
)

for chunk in agent.make_request(content='What were the latest AI innovations from Googol in 2024?'):
    print(chunk, end='')

# Callback Functions

### Built-in Callback Functions (StreamlitCallback)

In [ ]:
from cortex_agent.callbacks import StreamlitCallback

connection_parameters = {
    "account": "SFSEEUROPE-PROD-DEMO-GORKOW.snowflakecomputing.com",
    "user": "ADMIN",
    "role": "ACCOUNTADMIN",
    "warehouse": "COMPUTE_WH",
    "database": "cortex_agents_demo",
    "schema": "MAIN",
}

# Create agent
agent = CortexAgent(
    programmatic_access_token=agent_api_token, 
    connection_parameters=connection_parameters, 
    configuration=agent_config, 
    session=session
)

for chunk in agent.make_request(content='What was the total order quantity per month with status shipped? Create a bar plot.', callback=StreamlitCallback(agent)):
    pass

### Built-in Callback Functions (ConsolePrinter)

In [ ]:
from cortex_agent.callbacks import ConsoleCallback

connection_parameters = {
    "account": "SFSEEUROPE-PROD-DEMO-GORKOW.snowflakecomputing.com",
    "user": "ADMIN",
    "role": "ACCOUNTADMIN",
    "warehouse": "COMPUTE_WH",
    "database": "CORTEX_AGENTS_DEMO",
    "schema": "MAIN"
}

# Create agent
agent = CortexAgent(
    programmatic_access_token=agent_api_token, 
    connection_parameters=connection_parameters, 
    configuration=agent_config, 
    session=session
)

# Create the callback object
callback = ConsoleCallback(agent=agent, console_print=False)

for chunk in agent.make_request(content='What was the total order quantity per month with status shipped?', callback=callback):
    print(chunk)

### Built-in Callback Functions (ConversationalCallback)
Default callback function of Agent.

In [ ]:
from cortex_agent.callbacks import ConversationalCallback

# Create agent
agent = CortexAgent(
    programmatic_access_token=agent_api_token, 
    connection_parameters=connection_parameters, 
    configuration=agent_config, 
    session=session
)

# Create the callback object
callback = ConversationalCallback(agent=agent)

for chunk in agent.make_request(content='What were the latest AI innovations from Googol in 2024?', callback=callback):
    print(chunk, end='')

### Custom Callback Function
This callback function only prints User Messages and textual responses from Agent.

In [ ]:
from typing import Union
import json
from cortex_agent.message_formats import Message
from httpx_sse._models import ServerSentEvent

class MinimalisticCallback:
    def __init__(self):
        self.first_text_response = True

    def __call__(self, message: Union[Message, ServerSentEvent]):
        if isinstance(message, Message):
            response_string = f"User Message:\n{message['content'][0]['text']}\n\n"
            yield response_string
        if isinstance(message, ServerSentEvent):
            if message.event == "done":
                self.first_text_response = True
            if message.event == 'message.delta':
                data = json.loads(message.data)
                if data['delta']['content'][0]['type'] == 'text':
                    if self.first_text_response:
                        yield 'Assistant Message:\n'
                        self.first_text_response = False
                    yield data['delta']['content'][0]['text']

# Create agent
agent = CortexAgent(
    programmatic_access_token=agent_api_token, 
    connection_parameters=connection_parameters, 
    configuration=agent_config, 
    session=session
)

for chunk in agent.make_request(content='What were the latest AI innovations from Googol in 2024?', callback=MinimalisticCallback()):
    print(chunk, end='')

### Custom Callback Function (Raw Outputs)
This callback function returns raw outputs coming from Agent.

In [ ]:
import time 
import streamlit as st

def raw_callback(item):
    st.write(item)
    time.sleep(10)
    return item

agent_api_token = open('pat.txt').readline()
connection_parameters = {
    "account": "SFSEEUROPE-PROD-DEMO-GORKOW.snowflakecomputing.com",
    "user": "ADMIN",
    "role": "ACCOUNTADMIN",
    "warehouse": "COMPUTE_WH",
    "database": "CORTEX_AGENTS_DEMO",
    "schema": "MAIN"
}

# Create agent
agent = CortexAgent(
    programmatic_access_token=agent_api_token, 
    connection_parameters=connection_parameters, 
    configuration=agent_config, 
    session=session
)

for chunk in agent.make_request(content='What was the total order quantity per month with status shipped?', callback=raw_callback):
    pass